In [29]:
metadata = {
    "tf.keras.models.Sequential": {
        "sample_payload": {"name": "string"},
        "prerequisite": None
    },
    "tf.keras.layers.Dense": {
        "sample_payload": {"width": "int"},
        "prerequisite": "tf.keras.models.Sequential"
    },
    "model.compile": {
        "sample_payload": {"optimizer": "string", "loss": "string"},
        "prerequisite": "tf.keras.models.Sequential"
    },
    "model.fit": {
        "sample_payload": {"ep": "int"},
        "prerequisite": "model.compile"
    }
}


In [30]:
event_stream = [
    {"event": "user_message", "content": "I want to build a neural network."},
    {"event": "api_call", "api_name": "tf.keras.models.Sequential", "response": {"status": "success", "model_name": "this_model"}},
    {"event": "user_message", "content": "Let's add a dense layer."},
]

In [31]:
def create_sequential_model(name):
    # Simulate the API call to create a Sequential model
    print(f"API Call: tf.keras.models.Sequential(name='{name}')")
    response = {"status": "success", "model_name": name}
    event_stream.append({"event": "api_call", "api_name": "tf.keras.models.Sequential", "response": response})
    return response

def add_dense_layer(model_name, width):
    # Simulate the API call to add a Dense layer to the model
    print(f"API Call: tf.keras.layers.Dense(1, input_shape=[{width}]) to model {model_name}")
    response = {"status": "success", "layer": "Dense"}
    event_stream.append({"event": "api_call", "api_name": "tf.keras.layers.Dense", "response": response})
    return response

def compile_model(model_name, optimizer, loss):
    # Simulate the API call to compile the model
    print(f"API Call: {model_name}.compile(optimizer='{optimizer}', loss='{loss}')")
    response = {"status": "success", "compile": True}
    event_stream.append({"event": "api_call", "api_name": "model.compile", "response": response})
    return response

def fit_model(model_name, epochs):
    # Simulate the API call to fit the model
    print(f"API Call: {model_name}.fit(epochs={epochs})")
    response = {"status": "success", "fit": True}
    event_stream.append({"event": "api_call", "api_name": "model.fit", "response": response})
    return response

In [32]:
# Mapping of API names to their corresponding function implementations
api_function_mapping = {
    "tf.keras.models.Sequential": lambda payload: create_sequential_model(payload["name"]),
    "tf.keras.layers.Dense": lambda payload, model_name: add_dense_layer(model_name, int(payload["width"])),
    "model.compile": lambda payload, model_name: compile_model(model_name, payload["optimizer"], payload["loss"]),
    "model.fit": lambda payload, model_name: fit_model(model_name, int(payload["ep"]))
}


In [33]:
def find_in_event_stream(key):
    """
    Helper function to find a piece of information in the event stream.
    """
    for event in reversed(event_stream):  # Reverse to find the most recent occurrence
        if event.get("event") == "user_message" and key.lower() in event.get("content", "").lower():
            return event.get("content")
    return None

def check_prerequisite(api_name):
    """
    Check if the prerequisite API call for the given API has been made.
    """
    prerequisite = metadata[api_name]["prerequisite"]
    if not prerequisite:
        return True

    # Check if the prerequisite API call is in the event stream with a success status
    for event in event_stream:
        if event.get("event") == "api_call" and event.get("api_name") == prerequisite and event["response"].get("status") == "success":
            return True

    return False

In [34]:
def resolve_and_execute(api_name):
    """
    Resolve dependencies and execute the specified API call dynamically based on metadata.
    """
    if not check_prerequisite(api_name):
        print(f"Error: Prerequisite {metadata[api_name]['prerequisite']} not met for {api_name}.")
        return None

    # Resolve the payload for the API call
    payload = {}
    for key in metadata[api_name]["sample_payload"]:
        value = find_in_event_stream(key)
        if not value:
            value = input(f"Please provide {key}: ")
            event_stream.append({"event": "user_message", "content": f"My {key} is {value}."})
        payload[key] = value.split(" is ")[-1]

    # Check if the API name exists in the function mapping
    if api_name not in api_function_mapping:
        print(f"Unknown API call: {api_name}")
        return None

    # Execute the API call dynamically using the mapping and metadata
    if api_name == "tf.keras.models.Sequential":
        return api_function_mapping[api_name](payload)

    else:
        # Fetch model name or check previous steps dynamically
        prerequisite_api_name = metadata[api_name]["prerequisite"]
        model_event = next((event for event in event_stream if event.get("api_name") == prerequisite_api_name), None)

        if not model_event:
            print(f"Error: Prerequisite {prerequisite_api_name} not met or not initialized for {api_name}.")
            return None

        model_name = model_event["response"].get("model_name", "this_model")
        return api_function_mapping[api_name](payload, model_name)


In [35]:
# Agent decides to initialize a model
resolve_and_execute("tf.keras.models.Sequential")

# User wants to add a dense layer
resolve_and_execute("tf.keras.layers.Dense")

# Agent decides to compile the model
resolve_and_execute("model.compile")

# User wants to train the model
resolve_and_execute("model.fit")

Please provide name: model1
API Call: tf.keras.models.Sequential(name='model1')
Please provide width: 10
API Call: tf.keras.layers.Dense(1, input_shape=[10]) to model this_model
Please provide optimizer: adam
Please provide loss: mse
API Call: this_model.compile(optimizer='adam', loss='mse')
Please provide ep: 20
API Call: this_model.fit(epochs=20)


{'status': 'success', 'fit': True}